## Using customized model with python-mldb

You can add your own model into our development pipeline. It is easy to combine the model you want with python-mldb, thus utilize the advantages of MySQL database with machine learning application development.

### Dealer
You can access python-mldb with one object, Dealer. Dealer can access database, train model and save it to database, load model from database and use it to predict.  
When Dealer object establish, it can load *.csv file into database through dealer.dataset.
Dealer can start a training procedure with the dataset using dealer.procedure.train.

In [1]:
import os

from python_mldb import Dealer


path = os.path.abspath('./')
print(path)

/home/johnny/my_repo/python-mldb/Example


In [2]:
dealer = Dealer.Dealer(os.path.join(path, 'config_file/config.yaml'))

Connection established.
Query: SHOW DATABASES; done.
Query: USE test done.
Dealer established, service start!


### Dealer.dataset
#### Save into database
Now we will load .csv file into database. Here we use Titanic Competition data from Kaggle. You can download the file on the [competition page in Kaggle](https://www.kaggle.com/francksylla/titanic-machine-learning-from-disaster).
We load train.csv and test.csv using dealer.dataset.

In [3]:
data_path = os.path.join(path, '../data/')
os.path.exists(data_path)

train_data_name = 'TitanicTrain'
test_data_name = 'TitanicTest'

train_data_path = os.path.join(data_path, 'train.csv')
test_data_path = os.path.join(data_path, 'test.csv')

dealer.dataset.save_to_database(train_data_path, train_data_name)
dealer.dataset.save_to_database(test_data_path, test_data_name)

Query: SHOW TABLES; done.
Query: SHOW TABLES; done.


#### Load from database
Let's check the data we just save into database. Dealer.dataset.load_from_database will return a pandas.dataframe if the table you want to access exist.

In [4]:
train_data = dealer.dataset.load_from_database(train_data_name)
test_data = dealer.dataset.load_from_database(test_data_name)

Query: SHOW TABLES; done.
Query: SHOW COLUMNS FROM TitanicTrain done.
Query: SELECT * FROM TitanicTrain done.
Query: SHOW TABLES; done.
Query: SHOW COLUMNS FROM TitanicTest done.
Query: SELECT * FROM TitanicTest done.


In [5]:
print("Test data feture : \n", test_data.columns)
print("Train data feature \n: ", train_data.columns)
print("First 5 elements : \n")
n = 5
guests = test_data.head(5).values
for guest in guests:
    print(guest)

Test data feture : 
 Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Train data feature 
:  Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
First 5 elements : 

['892' '3' 'Kelly, Mr. James' 'male' '34.5' '0' '0' '330911' '7.8292'
 '0.0' 'Q']
['893' '3' 'Wilkes, Mrs. James (Ellen Needs)' 'female' '47.0' '1' '0'
 '363272' '7.0' '0.0' 'S']
['894' '2' 'Myles, Mr. Thomas Francis' 'male' '62.0' '0' '0' '240276'
 '9.6875' '0.0' 'Q']
['895' '3' 'Wirz, Mr. Albert' 'male' '27.0' '0' '0' '315154' '8.6625'
 '0.0' 'S']
['896' '3' 'Hirvonen, Mrs. Alexander (Helga E Lindqvist)' 'female' '22.0'
 '1' '1' '3101298' '12.2875' '0.0' 'S']


In [6]:
feature = ['PassengerId', 'Pclass', 
           'Age', 'SibSp', 'Parch']
label = ['Survived']

train_x = train_data[feature].values
train_y = train_data[label].values

test_x = test_data[feature].values

### Dealer.Procedure
#### Train using dealer.procedure with data saved into database
Register Procedure for dealer, for example here we use Keras binary classifier, which is a MLP model. Add to dealer's procedure_dict.
Here we add keras model into CustomizedProcedure.

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD


model = Sequential()
model.add(Dense(64, input_dim=5, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Using TensorFlow backend.


In [8]:
from python_mldb import Procedure


model_name = 'mlp_classifier' 
model_table_name = 'keras_binary_classifier'
mlp_classifier = Procedure.CustomizedClassifierProcedure(dealer.query_handler,
                                                         dealer.dataset,
                                                         model_name,
                                                         model_table_name,
                                                         model,
                                                         'fit',
                                                         'save')
dealer.procedure_dict[model_name] = mlp_classifier

In [9]:
procedure = dealer.procedure_dict[model_name]
procedure.train(train_data_name, label_col=label, feature_col=feature, epochs=100)

Query: SHOW TABLES; done.
Query: SHOW COLUMNS FROM TitanicTrain done.
Query: SELECT * FROM TitanicTrain done.
Start training mlp_classifier classifier (model type : <class 'keras.engine.sequential.Sequential'> ) with dataset TitanicTrain.
Epoch 1/100
891/891 [==============================] - 0s 163us/step - loss: 5.9586 - acc: 0.5948
Epoch 2/100
891/891 [==============================] - 0s 18us/step - loss: 5.7034 - acc: 0.6038
Epoch 3/100
891/891 [==============================] - 0s 18us/step - loss: 6.0956 - acc: 0.5600
Epoch 4/100
891/891 [==============================] - 0s 18us/step - loss: 5.5896 - acc: 0.6072
Epoch 5/100
891/891 [==============================] - 0s 18us/step - loss: 5.7859 - acc: 0.5859
Epoch 6/100
891/891 [==============================] - 0s 18us/step - loss: 5.7818 - acc: 0.5870
Epoch 7/100
891/891 [==============================] - 0s 17us/step - loss: 5.6262 - acc: 0.5836
Epoch 8/100
891/891 [==============================] - 0s 19us/step - loss: 5.719

### Dealer.Function

#### Reference
We can load the model we just train and saved in database using dealer.function. The process is like using procedure.

In [10]:
from python_mldb import Function
from keras.models import load_model


rf_classifier_func = Function.CustomizedClassifierFunction(dealer.query_handler, 
                                                           dealer.dataset, 
                                                           model_name, 
                                                           model_table_name,
                                                           'predict',
                                                           load_model)
dealer.function_dict[model_name] = rf_classifier_func

In [11]:
function = dealer.function_dict[model_name]
function.show_model(model_table_name)

Query: SHOW TABLES; done.
Query: SELECT * FROM keras_binary_classifier done.
('mlp_classifier', datetime.datetime(2018, 12, 21, 14, 51, 54), 'TitanicTrain', '/home/johnny/my_repo/python-mldb/saved_model/2018-12-21T14:51:54.350580_TitanicTrain_mlp_classifier.h5')
('mlp_classifier', datetime.datetime(2018, 12, 21, 14, 55, 15), 'TitanicTrain', '/home/johnny/my_repo/python-mldb/saved_model/2018-12-21T14:55:15.221400_TitanicTrain_mlp_classifier.h5')
('mlp_classifier', datetime.datetime(2018, 12, 21, 15, 1, 14), 'TitanicTrain', '/home/johnny/my_repo/python-mldb/saved_model/2018-12-21T15:01:14.366029_TitanicTrain_mlp_classifier.h5')
('mlp_classifier', datetime.datetime(2018, 12, 21, 15, 6, 52), 'TitanicTrain', '/home/johnny/my_repo/python-mldb/saved_model/2018-12-21T15:06:52.170064_TitanicTrain_mlp_classifier.h5')


In [12]:
function.reference(model_name, '2018-12-21 14:55:15', train_data_name, test_data_name, feature)

Query: SHOW TABLES; done.
Query: SHOW TABLES; done.
Query: SHOW COLUMNS FROM TitanicTest done.
Query: SELECT * FROM TitanicTest done.
Query: SELECT model_path FROM keras_binary_classifier WHERE name='mlp_classifier' AND savetime='2018-12-21 14:55:15' AND dataset='TitanicTrain' done.
418/418 [==============================] - 0s 62us/step
[[0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.4260663]
 [0.426066